<a href="https://colab.research.google.com/github/Shivam10816/CS22M082_ASSIGNEMNT_1/blob/q2/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import numpy as np
import random
from keras.datasets import fashion_mnist

In [ ]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [ ]:
def initialization(hi):
 
  l= len(train_data[0])*len(train_data[0])

  W =[np.matrix( tf.Variable(tf.random.truncated_normal([hi[0], l], stddev=1.0 / tf.sqrt(float(l)))))] 
  b =[np.random.randn(1,hi[0])]
  A =[np.zeros((1,hi[0]))]
  H = [np.zeros((1,hi[0]))]
  for i in range(1,len(hi)) :
    W.append(np.random.randn(hi[i],hi[i-1]))
    b.append(np.random.randn(1,hi[i]))
    A.append(np.zeros((1,hi[i])))
    H.append(np.zeros((1,hi[i])))
  W.append(np.random.randn(10,hi[-1]))
  A.append(np.zeros((1,10)))
  b.append(np.random.randn(1,10))
  return W , b ,A,H


In [ ]:
 print("-------W-----")
for i in W :
  print(i.shape)
print("-------b-----")
for i in b :
  print(i.shape)
print("-------A-----")
for i in A :
  print(i.shape)
print("-------H-----")
for i in H :
  print(i.shape)

-------W-----
(50, 784)
(50, 50)
(10, 50)
-------b-----
(1, 50)
(1, 50)
(1, 10)
-------A-----
(1, 50)
(1, 50)
(1, 10)
-------H-----
(1, 50)
(1, 50)


In [ ]:
def forward_pro(image):

      l = len(image)*len(image)
      x = np.reshape(image,(1,l)); # 1 x 784
      

      for i in range(0,hi[0]):
  
        
        A[0][0][i]=(np.matmul(W[0][i],x.transpose())+b[0][0][i]) # a0 = WoX +bo
        

      
      for i in range(1,len(hi)):
  
        H[i-1][0]=sigmoid(H[i-1][0],A[i-1][0]) # hi = g(ai)
        #print(H[i-1])
        for j in range(0,hi[i]):
          A[i][0][j]=(np.dot(W[i][j],H[i-1][0])+b[i][0][j])  # ai = WiX +bi


      H[-1][0]=sigmoid(H[-1][0],A[-2][0])

      for j in range(0,10):
          A[-1][0][j]=(np.dot(W[-1][j],H[-1][0])+b[-1][0][j])
      
      y_hat = softmax((A[-1][0]))
      #print(y_hat)
      return ((y_hat))
      # print("---data in W------")
      # for i in W :
      #   print(i)
      # print("---data in b------")
      # for i in b :
      #   print(i)
      # print("---data in A------")
      # for i in A :
      #   print(i)
      # print("---data in H------")
      # for i in H :
      #   print(i)

In [ ]:
def softmax(z):
      s = np.max(z)
      e_x = np.exp(z - s)
      div = np.sum(e_x)
      return e_x / div
    
   
def sigmoid(H,A):
  for i in range(0,len(A)):
      if A[i]<0 :
        H[i]= np.exp(A[i])/(1+np.exp(A[i]))
      else:
        H[i]=  1/(1+np.exp(-A[i]))
  return H

In [ ]:
def substract(W,W_theta,step_size):
      for i in range(0,len(W)):
        W[i]-=(step_size*W_theta[i])
      return dw

In [ ]:
hi=[50,50,50,50]
step_size=0.001
epoch =2
W,b,A,H =initialization(hi)
accuracy(test_data,test_labels)

accuracy :- 20.95 %


time required for 60000 image  222.55897521972656  sec


after 1 epoch accuracy :- 16.666666666666664 %
after 1 epoch accuracy :- 16.666666666666664 %


In [ ]:
train_data=train_data
test_data = test_data

In [ ]:
def sgd(step_size,batch_size,epoch):
      N = int(len(train_data)/batch_size)

      for e in range(0,epoch):
        start_time = time.time()
        for k in range(0,N):
          minibatch = train_data[k*batch_size:min(k*batch_size+batch_size,len(train_data))]
          minibatch_lable=train_labels[k*batch_size:min(k*batch_size+batch_size,len(train_data))]
          #print(minibatch.shape)
          y_hat=forward_pro(minibatch[0])
          dw,db = back_prop(minibatch[0],minibatch_lable[0],y_hat)

          for i in range(1,len(minibatch)):
            y_hat=forward_pro(minibatch[i])
            w_theta,b_theta = back_prop(minibatch[i],minibatch_lable[i],y_hat)
            for i in range(0,len(hi)+1):
              dw[i] +=w_theta[i]
              db[i] +=b_theta[i]
          
          for i in range(0,len(hi)+1):
            b[i] -=(step_size*db[i])
            W[i] -=(step_size*dw[i])
          W = substract(W,dw,step_size)
          b = substract(b,db,step_size)
        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def back_prop(image,lable,y_hat):

        l = len(image)**2
        W_theta , b_theta,H_theta,A_theta =W,b,H,A
        #print(np.argmax(y_hat),lable)
        ey = np.zeros((1,len(y_hat)))

        ey[0][lable]=1;
       
        L =len(A_theta)


        A_theta[L-1]=(-(ey-y_hat))
        
        #-------------------------
        for k in range(L-1,0,-1):
          
          W_theta[k]=((np.matmul(A_theta[k].transpose(),H[k-1]))) # athetak*h[k-1]
          b_theta[k]=A_theta[k]
          H_theta[k-1]=np.matmul(A_theta[k],W[k])
          
          g_z = A[k-1]
          a_k_1 = g_z
          g_z = sigmoid(g_z[0],A[k-1][0])
          # print("-------g_z------------")
          # print((g_z*(1-g_z)))
          A_theta[k-1]=H_theta[k-1]*(g_z*(1-g_z))

          

        x = np.reshape(image,(1,l));
        
        
        W_theta[0] = ((np.matmul(A_theta[0].transpose(),x)))
        b_theta[0]=A_theta[0]

        # print("-------W-----")
        # for i in W_theta :
        #   print(i)
        # print("-------b-----")
        # for i in b_theta :
        #   print(i)
        # print("-------A-----")
        # for i in A_theta :
        #   print(i)
        # print("-------H-----")
        # for i in H_theta :
        #   print(i)
        
        return W_theta , b_theta

epoch-  0
11.653964820454828
11.65628742637124
11.665061990990411
11.66114935002619
11.6620576757235
11.667565413846667
--- 4.168987035751343 seconds ---
epoch-  1
11.653956116855902
11.656278602651614
11.665052665736102
11.661140265760947
11.662048762124806
11.66755631101427
--- 4.923556566238403 seconds ---
epoch-  2
11.653947414226373
11.656269779905893
11.66504334149158
11.661131182489337
11.662039849508604
11.667547209180105
--- 4.056918144226074 seconds ---
epoch-  3
11.653938712566143
11.656260958133972
11.665034018256762
11.661122100211273
11.6620309378748
11.667538108344084
--- 4.217562437057495 seconds ---
epoch-  4
11.653930011875122
11.656252137335768
11.665024696031558
11.661113018926661
11.662022027223296
11.667529008506111
--- 4.658129453659058 seconds ---
epoch-  5
11.653921312153214
11.656243317511185
11.66501537481588
11.661103938635408
11.662013117554006
11.667519909666092
--- 4.29598069190979 seconds ---
epoch-  6
11.653912613400328
11.656234498660133
11.66500605460

TypeError: ignored